In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('p2_train.csv')
test = pd.read_csv('p2_test.csv')

In [3]:
train.head()

,Unnamed: 0,text,label
0,7744,longlost pictur archiv celebr glamour seneg wr...,0
1,10536,advanc understand natur world respons current ...,1
2,7043,itali test western economi bear almost total s...,1
3,5930,south africa box granni juke jab way healthier...,1
4,11660,coronaviru pandem nation tragedi hundr thousan...,1


In [4]:
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!pip install fastText

In [5]:
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')  # English
# ft = fasttext.load_model('cc.en.300.bin')

In [6]:
from gensim.models.fasttext import load_facebook_model

# Load FastText embeddings
ft_model = load_facebook_model('cc.en.300.bin')

In [7]:
X_train, X_test, y_train, y_test = train_test_split(train['text'], train['label'], test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Embedding, LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D, Attention
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# tokenization and padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
max_length = max([len(seq) for seq in X_train_seq])
max_length = max([len(seq) for seq in X_test_seq])
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')
print(max_length)

In [ ]:
embedding_dim = 300
# LSTM model with fasttext
model = Sequential()
model.add(Embedding(input_dim=len(ft_model.wv.key_to_index)+1,
                    output_dim=embedding_dim,
                    input_length=max_length,
                    weights=[np.vstack((np.zeros((1, embedding_dim)), ft_model.wv.vectors))],
                    trainable=False))
model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=5))
model.add(Dense(64, activation='tanh'))
model.add(GlobalMaxPooling1D())
model.add(Dense(2, activation='softmax'))  # 2 classes: generated by same method or not
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
model.fit(X_train_padded, y_train, epochs=15, batch_size=32)

Epoch 1/15
43/43 [==============================] - 13s 128ms/step - loss: 0.6680 - accuracy: 0.5587
Epoch 2/15
43/43 [==============================] - 5s 114ms/step - loss: 0.4882 - accuracy: 0.7309
Epoch 3/15
43/43 [==============================] - 5s 115ms/step - loss: 0.4373 - accuracy: 0.7837
Epoch 4/15
43/43 [==============================] - 5s 112ms/step - loss: 0.4262 - accuracy: 0.7852
Epoch 5/15
43/43 [==============================] - 5s 112ms/step - loss: 0.4017 - accuracy: 0.7830
Epoch 6/15
43/43 [==============================] - 5s 115ms/step - loss: 0.4046 - accuracy: 0.7962
Epoch 7/15
43/43 [==============================] - 5s 114ms/step - loss: 0.3831 - accuracy: 0.8043
Epoch 8/15
43/43 [==============================] - 5s 115ms/step - loss: 0.3871 - accuracy: 0.8028
Epoch 9/15
43/43 [==============================] - 5s 114ms/step - loss: 0.3628 - accuracy: 0.8218
Epoch 10/15
43/43 [==============================] - 5s 114ms/step - loss: 0.3725 - accuracy: 0.821

In [ ]:
tokenizer.fit_on_texts(X_test)
loss, acc = model.evaluate(X_test_padded, y_test)

11/11 [==============================] - 1s 53ms/step - loss: 0.4449 - accuracy: 0.8182


In [ ]:
preds = model.predict(X_test_padded)

11/11 [==============================] - 1s 51ms/step


In [ ]:
preds[0]

array([0.0046443, 0.9953557], dtype=float32)

In [ ]:
preds_new = []
for i in preds:
  if(i[0]>i[1]):
    preds_new.append(0.0)
  else:
    preds_new.append(1.0)

In [ ]:
preds_new[1]

1.0

In [ ]:
y_test_array = np.array(y_test)
y_test_array[1]

1

In [ ]:
from sklearn.metrics import f1_score,precision_score, recall_score, accuracy_score
f1 = f1_score(y_test, preds_new)
print("F1 score = ",f1)
precision = precision_score(y_test, preds_new)
recall = recall_score(y_test, preds_new)
print("Precision = ", precision)
print("Recall = ", recall)
print("Accuracy = ", accuracy_score(y_test, preds_new))

F1 score =  0.8187134502923976
Precision =  0.8333333333333334
Recall =  0.8045977011494253
Accuracy =  0.8181818181818182
